In [5]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
import pyvista as pv
import meshio
from pyvista import examples
import ipympl
from trame.widgets import vtk, vuetify
import subprocess
import sys

In [6]:
# constants
Length = 3.15
Height = 1.0
Nx = 50
Ny = 50

# material properties
nu = 0.12
Y = 0.2236

# nu = 0.60     # Poisson's ratio
# Y =  0.2236        # Young's modulus

# nu = 0.36    # Poisson's ratio
# Y = 0.0557       # Young's modulus

# nu = 0.12    # Poisson's ratio
# Y = 0.13965      # Young's modulus

mu = Constant(Y/(2*(1 + nu))) # shear modulus
lambda_ = Constant(Y*nu/((1 + nu)*(1 - 2*nu))) # Lame constant
# K = lambda_ * 2/3 *mu # bulk modulus

# Max displacement and displacement step for sweep
DEF_MAX = 1.2
dy = 0.02

for i in range(0,int(DEF_MAX/dy)+1):
    DEF_RATIO = round(i*dy,2)
    DISP = DEF_RATIO*Height

    out = "/home/test/Downloads/test/linear_f/out_" + str(DEF_RATIO)
    subprocess.run(["mkdir",out])

    mesh = RectangleMesh(Point(0., 0.), Point(Length, Height), Nx, Ny, "crossed")
    File("/home/test/Downloads/test/linear_f/my_mesh.xml") << mesh

    V = VectorFunctionSpace(mesh, 'CG', 1)

    def bottom(x, on_boundary):
        return (on_boundary and near(x[1], 0.0))

    def top(x, on_boundary):
        return (on_boundary and near(x[1], Height))

    # Define strain
    def epsilon(u):
        return 0.5*(nabla_grad(u) + nabla_grad(u).T)

    # Define stress
    def sigma(u):
        return lambda_*div(u)*Identity(u.geometric_dimension()) + 2*mu*epsilon(u)

    # Define Dirichlet boundary (y = 0, y = 1)
    b = Expression(("0.0", "0.0"), degree=1)
    t = Expression(("0.0", "H"), H=Height, degree=1)

    # Assign boundary conditions
    bcb = DirichletBC(V, b, bottom)
    bct = DirichletBC(V, t, top)
    bcs = [bcb, bct]

    # Define strain
    def epsilon(u):
        return 0.5*(nabla_grad(u) + nabla_grad(u).T)

    # Define stress
    def sigma(u):
        return lambda_*div(u)*Identity(u.geometric_dimension()) + 2*mu*epsilon(u)

    # Define variational problem
    u = TrialFunction(V)
    d = u.geometric_dimension()
    v = TestFunction(V)
    f = Constant((0, 0))
    T = Constant((0, 0))
    a = inner(sigma(u), epsilon(v))*dx
    L = dot(f, v)*dx + inner(T, v)*ds(1)

    # Compute solution
    u2 = Function(V)
    solve(a == L, u2, bcs)
    V2 = FunctionSpace(mesh, 'Lagrange', 1)
    # a, L = assemble_system(a, L, bcs)
    # solve(a, u.vector(), L)

    # Write to file
    X = V.tabulate_dof_coordinates()
    X.resize((V.dim(), 2))

    x = X[:,0]
    y = X[:,1]

    disp_out = open(out+"/disp.out","w")
    disp_field  = u2.vector().get_local()
     
    for it in range(0,int(disp_field.size/2)):
        # print(disp_field)
        # print(DEF_RATIO)
        # print(it)
        # print(str(x[2*it]) + ", " + str(y[2*it]) + ", " + str(disp_field[2*it]) + ", " + str(disp_field[2*it+1]) + "\n")
        disp_out.write(str(x[2*it]) + ", " + str(y[2*it]) + ", " + str(disp_field[2*it]) + ", " + str(disp_field[2*it+1]) + "\n")

    disp_out.close()

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational p